In [231]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pymc3 as pm
import theano.tensor as tt
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler

# Data manipulation
Importing data, splitting it to train and test (because of the imbalance between white and black water it would be better if we set a minimum of black points in the test and train)

In [129]:
otudf =pd.read_csv(filepath_or_buffer="/home/spanashis/Documents/Stats/Project/amazon-rivers/otudf",index_col=0)
wwfdf = pd.read_csv(filepath_or_buffer="/home/spanashis/Documents/Stats/Project/amazon-rivers/wwfdf",encoding="ISO-8859-1",index_col = "ID")
nmds20  = pd.read_csv(filepath_or_buffer="/home/spanashis/Documents/Stats/Project/amazon-rivers/nmds20dim",index_col=0)
# Black water sites. There are 21 black water sites and 143 white water
blackindex = wwfdf[wwfdf["Water"] == "Black"].index

# Splitting training and test data
np.random.seed(11235)
X_train,X_test,y_train,y_test = train_test_split(otudf,wwfdf.Water,test_size = 0.2)
np.random.seed(11235)
Xn_train,Xn_test,y_train,y_test = train_test_split(nmds20,wwfdf.Water,test_size = 0.2)

# Using Logistic Regression with Lasso
I am going to be trying both a frequentist and the bayesian approach. For the frequentist I will be using the sklearn package of python and for Bayesian I will use pymc3 to get the posterior distribution of the weights.

## Sklearn Frequentist Logistic regression

In [266]:
freq = LogisticRegression(penalty="l1",class_weight="balanced",multi_class="ovr",solver= "saga",max_iter=10000)
freq.fit(X_train,y_train)
freq.score(X_test,y_test)

0.9393939393939394

With saga solver and l1 penalty we get 94% accuracy on the test set
with liblinear it fluctuates between 85% and 90%, but it doesn't always converge

In [268]:
np.sort(freq.coef_)

array([[-1.16712035e-03, -8.85022297e-04, -8.74111074e-04,
        -7.78210729e-04, -7.76417990e-04, -7.06168423e-04,
        -5.05706379e-04, -4.90692464e-04, -4.77861073e-04,
        -4.26898301e-04, -4.04111602e-04, -3.99245712e-04,
        -3.78200407e-04, -3.19144610e-04, -3.07246344e-04,
        -2.72271370e-04, -2.50966370e-04, -2.45427072e-04,
        -2.42182202e-04, -2.23078358e-04, -1.94150415e-04,
        -1.91808101e-04, -1.71450474e-04, -1.62989869e-04,
        -1.52761619e-04, -1.33717603e-04, -1.12291243e-04,
        -1.10998956e-04, -1.09928252e-04, -1.06512031e-04,
        -1.05399908e-04, -1.04058807e-04, -1.01909792e-04,
        -1.01478045e-04, -9.74960272e-05, -9.29283369e-05,
        -9.06516177e-05, -8.97699763e-05, -8.30901108e-05,
        -7.77029370e-05, -7.14585551e-05, -7.05492475e-05,
        -6.92983826e-05, -6.47298329e-05, -5.26303206e-05,
        -5.09353275e-05, -4.84162435e-05, -4.77956843e-05,
        -4.69729041e-05, -4.55521320e-05, -4.46928885e-0

In [280]:
crossfreq =cross_val_score(freq,otudf,y=wwfdf.Water,cv=10,n_jobs = -1)

### NMDS 20 dimensions as features

In [264]:
freq = LogisticRegression(penalty="l1",class_weight="balanced",multi_class="ovr",solver= "liblinear",max_iter=10000)
freq.fit(Xn_train,y_train)
freq.score(Xn_test,y_test)

0.9090909090909091

In [265]:
freq.coef_

array([[ 1.63877858,  0.        ,  2.80635959,  0.66580615, -4.96560675,
        -1.01601738, -3.27691308,  0.        ,  0.        ,  0.        ,
         3.776655  ,  0.15222217,  0.        ,  0.68760929, -1.29608284,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

Both liblinear and saga produced 90% accuracy on the test set when using nmds as features

In [80]:
print(np.sum(y_test == "Black")/y_test.count())
np.sum(y_train == "Black")/(y_train.count())

0.12121212121212122


0.1297709923664122

In [90]:
 pd.DataFrame(data= freq.coef_.T).describe()
np.sum(freq.coef_ == 0)/np.size(freq.coef_)
# 32% of coefficients go to zero. The biggest weight has a magnitude of the order of 10e-4

0.3214814814814815

## Bayesian logistic regression using pymc3

I cant get it to work with either nmds or otutable, there might be something wrong with the code or with the implementation.

In [158]:
def logit(x):
    return(tt.exp(x)/(1+tt.exp(x)))

In [192]:
y_train.bool = (y_train == "White")*1
y_test.bool = (y_test == "White")*1

In [260]:
niter=2000
bay = pm.Model()
with bay:
    # Weights of the features have a laplace prior which is the same as l1 when minimising the loss function
    lam = 1
    beta = pm.Laplace("beta",0,lam,shape = (np.array(Xn_train).shape[1],1))
    inter = pm.Laplace("intercept", 0, lam)

    y_hat = tt.dot((Xn_train),beta) + inter
    # Calculating the probability of water being white p(y=1|mu) so that we can get 
    # p(y|x,beta) ~ Binomial(y|mu)
    mu =logit(y_hat)
    y_obs = pm.Binomial("Y_obs",n=np.ones(y_train.size),p = mu,observed = y_train.bool)
    trace = pm.sample(niter,random_seed=11235,step =pm.NUTS() )

Multiprocess sampling (2 chains in 2 jobs)
NUTS: [intercept, beta]
Sampling 2 chains: 100%|██████████| 5000/5000 [00:15<00:00, 327.72draws/s]


In [263]:
df_trace = pm.trace_to_dataframe(trace[niter//2:])
betas =df_trace.mean(0)[0:-1]
intercept = df_trace.mean(0)[-1]
pm.find_MAP(model=bay)
#bay.Y_obs

/home/spanashis/anaconda2/envs/py36/lib/python3.6/site-packages/pymc3/tuning/starting.py:61: UserWarning: find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.
  warnings.warn('find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.')
logp = -6,639.8, ||grad|| = 4.657: 100%|██████████| 37/37 [00:00<00:00, 729.30it/s]    


{'beta': array([[-6.62310080e-06],
        [ 4.16495878e-05],
        [-8.97076413e-06],
        [-5.26131853e-05],
        [ 9.62803841e-06],
        [-7.40750960e-05],
        [ 2.24612320e-05],
        [ 1.61002264e-05],
        [ 1.80049943e-05],
        [ 2.34860407e-05],
        [ 1.12658329e-05],
        [-2.22524733e-05],
        [ 1.29637215e-05],
        [ 1.00649396e-05],
        [ 4.59244067e-07],
        [ 4.94862295e-05],
        [-1.78496617e-05],
        [ 8.55649367e-06],
        [-1.12016865e-05],
        [-2.29966591e-05]]), 'intercept': array(1.9024686)}

In [253]:
def predict(X,betas=betas,intercept=intercept):
    
    v = np.dot(X,betas) + intercept
    return(np.exp(v)/(1+np.exp(v)))

In [254]:
(predict(Xn_train) >= 0.5 )*1 ==y_train.bool

ID
3-S20b     True
1-S02C     True
3-S12b     True
2-S17C     True
2-S15B    False
2-S16B     True
2-S15E    False
1-S03C     True
3-S14f     True
2-S19C     True
3-S01b     True
3-S18a     True
3-S18d     True
3-S10d    False
3-S06c     True
3-S21d     True
3-S17d     True
1-S06B     True
3-S20a     True
3-S21a     True
3-S16a     True
1-S05B     True
2-S13A    False
3-S05b     True
3-S11d    False
1-S02A     True
2-S15C    False
2-S14A    False
2-S18A    False
3-S20c     True
          ...  
1-S08B     True
3-S18c     True
3-S07c     True
2-S15A    False
3-S09c    False
2-S14C     True
1-S10A     True
3-S12c     True
3-S17A     True
3-S04a     True
2-S12A     True
3-S03b     True
2-S13D     True
3-S03c     True
3-S04d     True
3-S15c     True
1-S05C     True
3-S03d     True
3-S19c     True
3-S01c     True
1-S11C     True
3-S09b     True
3-S14g     True
3-S15b     True
2-S14B     True
1-S04B     True
2-S17B     True
3-S17b     True
3-S11b     True
3-S20d    False
Name: Water, Length: 

In [53]:
n = 5 * np.ones(4)
x = np.array([-0.896, -0.296, -0.053, 0.727])
y = np.array([0, 1, 3, 5])

In [157]:
niter=1000
def invlogit(x):
    return tt.exp(x) / (1 + tt.exp(x))

with pm.Model() as model:
    alpha = pm.Laplace('alpha', mu=0, b=1)
    beta = pm.Laplace('beta',0,1)

    p = invlogit(alpha + tt.dot(beta,x))
    y_obs = pm.Binomial('y_obs', n=n, p=p, observed=y)

    trace = pm.sample(niter, random_seed=123)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [beta, alpha]
Sampling 2 chains: 100%|██████████| 3000/3000 [00:02<00:00, 1444.81draws/s]


In [135]:
for RV in bay.basic_RVs:
    print(RV.name, RV.logp(bay.test_point))

beta -13.862943611198906
intercept -0.6931471805599453
Y_obs -11895.098765591549


## Random Forrest

In [199]:
def rfr_model(X, y):
# Perform Grid-Search
    gsc = GridSearchCV(
        estimator=RandomForestClassifier(),
        param_grid={
            'max_depth': range(3,7),
            'n_estimators': (500, 1000,2000),
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0,                         n_jobs=-1)
    
    grid_result = gsc.fit(X, y)
    best_params = grid_result.best_params_
    
    #rfr = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"],random_state=False, verbose=False)
# Perform K-Fold CV
    #scores = cross_val_score(rfr, X, y, cv=10, scoring='neg_mean_absolute_error')

    return best_params

In [200]:
bestpar =rfr_model(X_train,y_train.bool)

/home/spanashis/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [213]:
rfr =RandomForestClassifier(max_depth=None, n_estimators=10000,random_state=True, verbose=True)
rfr.fit(X_train,y_train)
rfr.score(X_test,y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 10000 out of 10000 | elapsed:    4.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 10000 out of 10000 | elapsed:    0.9s finished


0.9393939393939394

Random forest with 10000 trees, max depth set to None and random_state set to True produces an accuracy of 93%, which is equivalent to that of the logistic regression with the saga solver.

In [215]:
rfr.fit(Xn_train,y_train)
rfr.score(Xn_test,y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 10000 out of 10000 | elapsed:    4.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 10000 out of 10000 | elapsed:    0.7s finished


0.9696969696969697

Using the nmds axis we get an accuracy of 96.7% (which is a single mistake!)

In [204]:
sum(rfr.predict(X_test) == y_test)/y_test.size

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.1s finished


0.9090909090909091

In [274]:
crossrfr =cross_val_score(rfr,otudf,y=wwfdf.Water,cv=10)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 10000 out of 10000 | elapsed:    5.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 10000 out of 10000 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 10000 out of 10000 | elapsed:    5.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 10000 out of 10000 | elapsed:    1.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 10000 out of 10000 | elapsed:    5.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 10000 out of 10000 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurren